In [1]:
import subprocess
import os
from data_utils import sentence_to_list_without_stopwords

input = 'tumalon ang unggoy sa mapuno'
input = ' '.join(sentence_to_list_without_stopwords(input))

p1 = subprocess.run(['java', '-jar', '--enable-preview', 'fspost-lemma.jar', input], capture_output=True, text=True)
output = p1.stdout.split('\n')[3:-1]
tag = output[0].split()
word = output[1].split()
[[(word,tag) for word,tag in zip(word,tag)]]

[[('talon', 'VBAF'), ('unggoy', 'NNC'), ('puno', 'VBW')]]

# Make sure to finish the steps in 04-EDA.ipynb

In [2]:
import pandas as pd
df_semantics_predefined = pd.read_csv(r'dataset\doccano-annotated\complete-unique-label-eda-collapsed.csv')
df_semantics_predefined.head()

,tokens,tags
0,taba,Perception_Feel_Emotion
1,taboy,Communication
2,taboy,Contact_Interact
3,tag,Objects
4,taga,Humans_Body


# Converting POS Tag to Semantics Tag

In [14]:
verb = ['Motion','Change','Contact','Communication','Perception_Feel_Emotion']
test = "Contact_Interact"
any([v for v in verb if v in test])


True

In [16]:
from ast import literal_eval
import pandas as pd

verb = ['Motion','Change','Contact','Communication','Perception_Feel_Emotion']
sentence_semantics = []
df_semantics = pd.read_csv(r'dataset\annotation\tl-storytelling-pos-tagged-lemmatized.csv')
for tokens,tags in zip(df_semantics['tokens'],df_semantics['tags']):
    semantic_tags = []
    for token,tag in zip(literal_eval(tokens),literal_eval(tags)):
        index = df_semantics_predefined[df_semantics_predefined['tokens'] == token].index.values
        detected_values = df_semantics_predefined.loc[index]['tags'].values
        if len(detected_values): 
            if any([v for v in verb if v in detected_values[0]]):
                semantic_tags.append("O")

            # if len(detected_values) > 1:
            #     if 'VB' in tag:
            #         # verb_semantic = [x for x in detected_values if x in verb]
            #         semantic_tags.append(verb_semantic)
            else:
                semantic_tags.append(detected_values[0])
        else:
            semantic_tags.append("O")
    sentence_semantics.append(semantic_tags)
df_semantics['ner_tags'] = sentence_semantics
df_semantics.head()

,tokens,tags,ner_tags
0,"['alitaptap', 'alitaptap', 'halina', 'halina',...","['FW', 'FW', 'FW', 'FW', 'DTCP', 'NNC_CCP', 'V...","[Animals, Animals, O, O, O, O, O, O, O, Natura..."
1,"['alitaptap', 'alitaptap', 'halina', 'halina',...","['FW', 'FW', 'FW', 'FW', 'RBW', 'VBTF']","[Animals, Animals, O, O, Natural_Environment, O]"
2,"['darating', 'umaga']","['VBTF', 'RBW']","[O, Natural_Environment]"
3,"['halina', 'huwag', 'takot', 'dilim', 'alitapt...","['RBW', 'RBF', 'VBW', 'NNC', 'NNC', 'NNC', 'NN...","[O, O, O, Natural_Environment, Animals, Animal..."
4,"['halina', 'aliw']","['RBW', 'NNC']","[O, O]"


In [3]:
import pandas as pd

df = pd.read_csv('huggingface-autotrain-dataset-batch1.csv')
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,tokens,tags,ner_tags
0,"['kaniya', 'sangag', 'sarap']","['PRSP_CCP', 'VBTS', 'JJD']","['O', 'O', 'O']"
1,"['balik', 'srey', 'pov', 'kaniya', 'nayon', 'b...","['VBAF', 'FW', 'FW', 'PRSP_CCP', 'NNC', 'VBTS'...","['O', 'O', 'O', 'O', 'Urban_Environment', 'O',..."
2,"['kukunin', 'rin', 'saya', 'puchku']","['VBTF_VBOF', 'RBI', 'JJD_CCP', 'NNC']","['O', 'O', 'Objects', 'O']"
3,"['rona', 'halika', 'yaya', 'tatay']","['FW', 'NNC', 'NNC', 'NNC']","['O', 'O', 'Humans_Body', 'Humans_Body']"
4,"['pasok', 'tubig', 'bangka', 'kongkea']","['VBAF', 'NNC', 'NNC', 'NNC']","['O', 'Natural_Environment', 'Transportation',..."


In [6]:
df_train = df[:3000]
df_test = df[3000:]
df_train

,tokens,tags,ner_tags
0,"['kaniya', 'sangag', 'sarap']","['PRSP_CCP', 'VBTS', 'JJD']","['O', 'O', 'O']"
1,"['balik', 'srey', 'pov', 'kaniya', 'nayon', 'b...","['VBAF', 'FW', 'FW', 'PRSP_CCP', 'NNC', 'VBTS'...","['O', 'O', 'O', 'O', 'Urban_Environment', 'O',..."
2,"['kukunin', 'rin', 'saya', 'puchku']","['VBTF_VBOF', 'RBI', 'JJD_CCP', 'NNC']","['O', 'O', 'Objects', 'O']"
3,"['rona', 'halika', 'yaya', 'tatay']","['FW', 'NNC', 'NNC', 'NNC']","['O', 'O', 'Humans_Body', 'Humans_Body']"
4,"['pasok', 'tubig', 'bangka', 'kongkea']","['VBAF', 'NNC', 'NNC', 'NNC']","['O', 'Natural_Environment', 'Transportation',..."
...,...,...,...
2995,"['nang', 'uwi', 'iyak']","['RBW', 'VBAF', 'VBTR_VBAF']","['O', 'O', 'O']"
2996,"['aalis', 'ba', 'mamaya', 'aalis', 'ba', 'ngay...","['VBTF', 'RBI', 'RBW', 'VBTF', 'RBI', 'RBW_CCP...","['O', 'O', 'O', 'O', 'O', 'O', 'Natural_Enviro..."
2997,"['nakiusap', 'doktor', 'kay', 'ira', 'turok', ...","['VBTS', 'NNC', 'DTP', 'FW', 'VBAF', 'DTP', 'N...","['O', 'Humans_Body', 'O', 'O', 'O', 'O', 'Anim..."
2998,"['baba', 'tatay', 'kay', 'rona']","['VBAF', 'NNC', 'DTP', 'NNC']","['O', 'Humans_Body', 'O', 'O']"


In [7]:
df_train.to_csv('huggingface-autotrain-dataset-train.csv', index=False)

# Pre-defining dataset with labeled unique words

In [ ]:
import pandas as pd
from ast import literal_eval 

# create an empty list to create a list of dictionaries
list_json = []

for x, i in enumerate(sentences):
    i = literal_eval(i) #Parsing string array to array
    list_json.append({"id": x}) # create id key for sentence
    list_json[x]["text"] = ' '.join(i) # create the text key for the sentence
    list_json[x]["label"] = [] # create an empty label key to later append
    for y, j in enumerate(labels[x]):
        if j in unique_labels:
            word = i[y] # determine what the word is
            # 
            # If I don't add an if statement here, the label indexes for the first always got tagged 1 index to the right
            # I am not sure why but this if statement fixes the problem
            #
            if y == 0: 
                # determining the start and end index of the label in the sentence
                wordStartIndex1 = len(" ".join(i[0:y])) # join word elements with a space until the word index to determine the start of the label index
                wordEndIndex1 = len(" ".join(i[0:y]))+ len(word) # add the length of the word to determine the End index
                list_json[x]["label"].append([wordStartIndex1, wordEndIndex1, j])
            # same thing but if its not the first of the sentence, we add 1 to the start and end indexes
            else:
                wordStartIndex1 = len(" ".join(i[0:y]))+1
                wordEndIndex1 = len(" ".join(i[0:y]))+ len(word)+1
                list_json[x]["label"].append([wordStartIndex1, wordEndIndex1, j])
list_json[0:5]

In [5]:
# you can just dump a list of dictionaries as a json1 file
# which is the format required by doccano
import json
with open('predefined-dataset-batch1.json1', 'w') as f:
    for d in list_json:
        json.dump(d, f)
        f.write('\n')